In [26]:
#importing libraries
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from summarytools import dfSummary
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

In [2]:
#readind the data 
titanic = sns.load_dataset("titanic")
print(titanic.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')


In [3]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,Second,man,True,NaN,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,S,First,woman,False,B,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,S,Third,woman,False,NaN,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,C,First,man,True,C,Cherbourg,yes,True


In [4]:
dfSummary(titanic, is_collapsible=False)

No,Variable,Stats / Values,Freqs / (% of Valid),Graph,Missing
1,survived[int64],Mean (sd) : 0.4 (0.5)min < med < max:0.0 < 0.0 < 1.0IQR (CV) : 1.0 (0.8),2 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAAYBJREFUeJzt3UFqwkAYhuGZIqKbBBE8jIfoYXsIbyNlXFQpaLrrplBjVNoPnmc985PFS0IWmdRhGAr8dy9/fQEwxuzaglrrspQynzj/cxiG48S98O3XUGuty/V6/dp13WrK8MPh8F5rfRMr97p2R513XbfabrfHvu9PtwxurS12u91qv9/PSylC5S5XH/2llNL3/Wmz2XxMmL+csAd+8DJFBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESoRRX6HCGM88rESoPMSzDysRKo/y1MNKhMpDPeuwEi9TRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEUb9vqe1trh1cGttcT6fZ6WUrtZ684URp7tcLrOprVxb8wX9Y059YHhjbwAAAABJRU5ErkJggg=="">",0(0.0%)
2,pclass[int64],Mean (sd) : 2.3 (0.8)min < med < max:1.0 < 3.0 < 3.0IQR (CV) : 1.0 (2.8),3 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAAZJJREFUeJzt3VFqIkEYhdGqQURfuhHBxbiIWewswt3IUD5MwoD2rCCxNV1kbnLOa1tSNh9SCvrXaZoK/O9+fPYGYI7VZ2+Ar6PWui2lrJ9c/neappe3LgqVRdRat/v9/ucwDLtn1l8ul9+11l9vxSpUlrIehmF3PB5fxnF8fWRha21zOp125/N5XUoRKv2N4/h6OBz+PLF0+95FH6aIIFQiCJUIQiWCUIkgVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUIQiWCUIkgVCIIlQhCJYJQiSBUIgiVCEIlglCJIFQiCJUIQiWCUIkgVCIIlQhCJYJQiSBUItz9a/Seky5grndD7T3pAua6947addIFzDVrKkqvSRcwl/E9HTjXL0+oC3Ou70Ooy3Ou70ConTjXL8sX/kQQKhGESgShEkGoRBAqEYRKBKESQahEECoRhEoEoRJBqEQQKhGESgShEkGoRBAqEYRKBKESQahEmPUr1Nba5tEnbq1trtfrqpQy1Fof3liw4Xa7rb7hPfvQ6773mH+C3HcCikewqgAAAABJRU5ErkJggg=="">",0(0.0%)
3,sex[object],1. male2. female,577 (64.8%)314 (35.2%),"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAJsAAAAuCAYAAAA/ZmtKAAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAATlJREFUeJzt28Fpw0AURdE3QZhko0EIVIyKSLEpQt0IoVnExpvJwpAO/MbzfU8F3/hibKOXaq0CHJKkL0mX1ofg373Wem19xDMM8zx/j+M4tT4ED6WUI6X0EzG4YRzHaV3Xa8751vqYd3ee5+e2bdO+7xdJ8WKTpJzzbVmW39bHQNLja01IH60PwPsgNtgQG2yIDTbEBhtigw2xwYbYYENssCE22BAbbIgNNsQGG2KDDbHBZpAeD+21PgTx34ehlHJs2zYp8EN7PSmlHJLure94BgYvryfs4CUx5YNL9E+2sJ8SPQo95Ys8i+tR2Clf9Flcj6JP+fiF/UL4Uxc2xAYbYoMNscGG2GBDbLAhNtgQG2yIDTbEBhtigw2xwYbYYENssCE22ISd8kV8Tb0LPeWLPIvrEYMX2PwBCltq38NzG/wAAAAASUVORK5CYII="">",0(0.0%)
4,age[float64],Mean (sd) : 29.7 (14.5)min < med < max:0.4 < 28.0 < 80.0IQR (CV) : 17.9 (2.0),88 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb24zLjEwLjAsIGh0dHBzOi8vbWF0cGxvdGxpYi5vcmcvlHJYcgAAAAlwSFlzAAAPYQAAD2EBqD+naQAAAoNJREFUeJzt3UGO2jAUgOHnZkABpESICnZcoFIXLDlED9sjFIkTcAN2SFEUREpECOmm001nGOIkxc/9v72FF78gMbZs6roWwHWfnj0B4BEvz57APcaYkYgMLYdf6ro+dzkfPI+zoRpjRrPZ7FsURVOb8cfjMTXGfCdWPzgbqogMoyiartfrcxzHRZOBWZaF2+12miTJUEQI1QMuhyoiInEcF/P5/GfTcVVVRSISGWNsP5pHB4c4H6qNPM8HZVl+XS6XQRAEjb6NX/Ho4BYvQ71cLi9hGI5Xq1WxWCzSpuN5dHCPl6G+mkwmVo8Nv406nQxaYR0VKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSoQKlQgVKhAqFCBUKECoUIFQoUKhAoVvN443UZVVQOxP3PFeauOEeob2p654rxV9wj1DW3OXHHeqh+EekeLM1ect+oYL1NQgVChAqFCBUKFCoQKFQgVKhAqVCBUqECoUIFQoQKhQgVChQqEChUIFSqwza8HnA7oHqF2jNMB/SDUjnE6oB+E2hNOB3SLlymoQKhQ4cOffq4ihwvuhspV5HDFR9+oXEUOJzz01v+kq8ij2+02sBmoWcs/C0Q8fdzqbXmq7cJ3WZZhVVVfiqL4ISK2F++qwvXt7+st1LZXke/3++lutxtfr9egj/m5iOvb39f7gr/twneapv/twjfXt/+NdVSoQKhQgVChAqFCBXZPecbXTduE6hGfN20Tqkd83rRNqB7ycdM2L1NQgVChAj/9+MPlnVuEChFxf+cWoUJEutm5tdls5kmSfDbGHG2mcC/wh0LNsixs+qmn0ykUEcnzPDwcDuN/OV7rZ7swb1vn87nXNdxfmjKyuaZj420AAAAASUVORK5CYII="">",177(19.9%)
5,sibsp[int64],Mean (sd) : 0.5 (1.1)min < med < max:0.0 < 0.0 < 8.0IQR (CV) : 1.0 (0.5),7 distinct values,"<img src = ""data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAKoAAABGCAYAAABc8A97AAAAOnRFWHRTb2Z0d2FyZQBNYXRwbG90bGliIHZlcnNpb2

In [6]:
#We see these have missing values 
#deck, embark_town,embarked,age and duplicates
print(titanic.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'class', 'who', 'adult_male', 'deck', 'embark_town',
       'alive', 'alone'],
      dtype='object')


Preprocessing data

In [7]:
#dropping some columns 
titanic.drop(columns=['deck', 'embark_town', 'alive', 'who', 'class', 'adult_male'], inplace=True, errors='ignore')

In [8]:
print(titanic.columns)

Index(['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare',
       'embarked', 'alone'],
      dtype='object')


In [10]:
#Filling the missing values 
titanic['age'] = titanic['age'].fillna(titanic['age'].median())
titanic['embarked'] = titanic['embarked'].fillna(titanic['embarked'].mode()[0])

In [11]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,male,22.0,1,0,7.2500,S,False
1,1,1,female,38.0,1,0,71.2833,C,False
2,1,3,female,26.0,0,0,7.9250,S,True
3,1,1,female,35.0,1,0,53.1000,S,False
4,0,3,male,35.0,0,0,8.0500,S,True
...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S,True
887,1,1,female,19.0,0,0,30.0000,S,True
888,0,3,female,28.0,1,2,23.4500,S,False
889,1,1,male,26.0,0,0,30.0000,C,True


In [12]:
#removing duplicates 
titanic = titanic.drop_duplicates()

In [16]:
# Encode categorical variables
titanic.loc[:, 'sex'] = label_encoder.fit_transform(titanic['sex']).astype(int)
titanic.loc[:, 'embarked'] = label_encoder.fit_transform(titanic['embarked']).astype(int)

In [17]:
titanic

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,1,22.0,1,0,7.2500,2,False
1,1,1,0,38.0,1,0,71.2833,0,False
2,1,3,0,26.0,0,0,7.9250,2,True
3,1,1,0,35.0,1,0,53.1000,2,False
4,0,3,1,35.0,0,0,8.0500,2,True
...,...,...,...,...,...,...,...,...,...
885,0,3,0,39.0,0,5,29.1250,1,False
887,1,1,0,19.0,0,0,30.0000,2,True
888,0,3,0,28.0,1,2,23.4500,2,False
889,1,1,1,26.0,0,0,30.0000,0,True


In [19]:
# Convert boolean values to integers
titanic.loc[:, 'alone'] = titanic['alone'].astype(int)

In [20]:
# Define features (X) and target variable (y)
X = titanic.drop(columns=['survived'])
y = titanic['survived']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Normalize Features
Since neural networks perform better with scaled data, we standardize the numeric features.

In [22]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Build the ANN Model

In [23]:
def build_model():
    model = keras.Sequential([
        layers.Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
        layers.Dropout(0.2),  # Dropout to prevent overfitting
        layers.Dense(16, activation='relu'),
        layers.Dense(8, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Sigmoid activation for binary classification
    ])

    model.compile(optimizer='adam', 
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

model = build_model()


Train the Model with Early Stopping

In [24]:
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

history = model.fit(X_train_scaled, y_train, 
                    validation_data=(X_test_scaled, y_test),
                    epochs=100, batch_size=32, 
                    callbacks=[early_stopping])

Epoch 1/100
20/20 [==============================] - 1s 17ms/step - loss: 0.6993 - accuracy: 0.5258 - val_loss: 0.6640 - val_accuracy: 0.5806
Epoch 2/100
20/20 [==============================] - 0s 4ms/step - loss: 0.6668 - accuracy: 0.5871 - val_loss: 0.6238 - val_accuracy: 0.6258
Epoch 3/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6411 - accuracy: 0.6194 - val_loss: 0.5863 - val_accuracy: 0.6710
Epoch 4/100
20/20 [==============================] - 0s 3ms/step - loss: 0.6159 - accuracy: 0.6935 - val_loss: 0.5602 - val_accuracy: 0.7742
Epoch 5/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5996 - accuracy: 0.7129 - val_loss: 0.5336 - val_accuracy: 0.8194
Epoch 6/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5658 - accuracy: 0.7661 - val_loss: 0.5005 - val_accuracy: 0.8258
Epoch 7/100
20/20 [==============================] - 0s 3ms/step - loss: 0.5546 - accuracy: 0.7468 - val_loss: 0.4810 - val_accuracy: 0.8258
Epoch 8/100


Evaluate the Model

In [25]:
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

5/5 [==============================] - 0s 2ms/step - loss: 0.4139 - accuracy: 0.8258
Test Loss: 0.4139, Test Accuracy: 0.8258


Loss (0.4139): A lower loss generally indicates a better model fit, though the optimal value depends on the problem and the loss function used (e.g., cross-entropy for classification).

Accuracy (82.58%): This suggests the model correctly predicted around 83% of the test data, which is a decent result depending on the complexity of the problem.

### Analyzing misclassifications to help identify where the model struggles.

In [27]:
# Assuming y_test are true labels and y_pred are model predictions
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)  # Convert probabilities to class labels

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_classes))

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))

5/5 [==============================] - 0s 2ms/step
Confusion Matrix:
[[95  0]
 [60  0]]

Classification Report:
              precision    recall  f1-score   support

           0       0.61      1.00      0.76        95
           1       0.00      0.00      0.00        60

    accuracy                           0.61       155
   macro avg       0.31      0.50      0.38       155
weighted avg       0.38      0.61      0.47       155



C:\Users\S P E C T R E\anaconda3\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\S P E C T R E\anaconda3\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\S P E C T R E\anaconda3\envs\dl\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(r

Severe Class Imbalance:
The model predicts only class 0, meaning it never identifies class 1.

Precision, recall, and F1-score for class 1 are 0.00, which is a red flag.

Bias Toward the Majority Class:
If class 0 is much more frequent than class 1, the model learns to always predict 0 because it minimizes overall error.

Even though accuracy is 61%, it’s misleading because the model isn’t learning class 1 at all.